Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [ ]:
DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'

import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [ ]:
import numpy as np
# wrangle the data to get going
def wrangle(X):
  X = X.copy()
  # set the index to train and test
  X['latitude'] = X['latitude'].replace(-2e-08, 0)
  cols_with_zeros = ['longitude', 'latitude', 'construction_year',
                       'gps_height', 'population']
  for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()
    # drop the duplicate columns
  duplicates = ['quantity_group', 'payment_type']
  X = X.drop(columns=duplicates)

  # Drop recorded by (never varies) and id(always varies, random)
  unusable_variance =['recorded_by', 'id']
  X = X.drop(columns = unusable_variance)

  #convert date into datetime object
  X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)

  #Extract components from date_recorded then drop the original column
  X['year_recorded'] = X['date_recorded'].dt.year
  X['month_recorded'] = X['date_recorded'].dt.month
  X['day_recorded'] = X['date_recorded'].dt.day
  X = X.drop(columns = 'date_recorded')

  #Engineer the features: how many years from construction_year to date_recorded
  X['years'] = X['year_recorded'] - X['construction_year']
  X['year_MISSING'] = X['years'].isnull()

  #return the wrangled dataframe
  return X

In [ ]:
# wrangle the train and test data
train = wrangle(train)
test = wrangle(test)

### Removing high cardinality features

In [ ]:
# target and features selection
target = 'status_group'

# get a dataframe with all train columns except the target
train_features = train.drop(columns=[target])

# get a list of numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

#get a series with the cardinality of the non numeric function
cardinality = train_features.select_dtypes(exclude='number').nunique()

#get a lit of all cardinal features with carinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# combine the lists
features = numeric_features + categorical_features

## Using Random Forest and RandomizedSearchCV using OHE

In [ ]:
#arrange the data into X features and y target vector for OneHotEncoder

X_train = train[features]
y_train = train[target]
X_test = test[features]

#arrange the data into X features and y target vector for OrdinalEncoder
#including all the features under OrdinalEncoder since it can handle any number of categories
#therefore no subsetting is required like the one above
X_train_OE = train.drop(columns=['status_group'])
X_test_OE = test

In [ ]:
#using OHE
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score

#making the pipeline
rsearchcv_pipeline_OHE = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_jobs=-1)
)
#defining the param_distribution for RandomizedSearchCV
param_distribution = {
    'simpleimputer__strategy':['mean', 'median'],
    'randomforestclassifier__n_estimators':range(100,500,100),
    'randomforestclassifier__max_depth':range(5,31,5),
    'randomforestclassifier__max_samples':np.arange(0.2,0.7,0.2),
}

search_model_OHE = RandomizedSearchCV(
    rsearchcv_pipeline_OHE,
    param_distributions=param_distribution,
    cv=5,
    n_jobs=-1,
    verbose=1,
    n_iter=10
)

In [ ]:
# fit the model
search_model_OHE.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  3.8min finished
/Users/rob/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('onehotencoder',
                                              OneHotEncoder(use_cat_names=True)),
                                             ('simpleimputer', SimpleImputer()),
                                             ('randomforestclassifier',
                                              RandomForestClassifier(n_jobs=-1,
                                                                     random_state=42))]),
                   n_jobs=-1,
                   param_distributions={'randomforestclassifier__max_depth': range(5, 31, 5),
                                        'randomforestclassifier__max_samples': array([0.2, 0.4, 0.6]),
                                        'randomforestclassifier__n_estimators': range(100, 500, 100),
                                        'simpleimputer__strategy': ['mean',
                                                                    'median']},
                   ver

In [ ]:
#print the scores
#scores
print('Best Hyperparameters', search_model_OHE.best_params_)
print(f'Best score with OHE and Random Forest is : {search_model_OHE.best_score_}')

Best Hyperparameters {'simpleimputer__strategy': 'median', 'randomforestclassifier__n_estimators': 300, 'randomforestclassifier__max_samples': 0.4, 'randomforestclassifier__max_depth': 25}
Best score with OHE and Random Forest is : 0.8112121212121212


In [ ]:
# assigning the best estimator values to the search_model_OHE
pipeline_OHE = search_model_OHE.best_estimator_
print(f'Training score with OHE and Random Forest is  {pipeline_OHE.score(X_train,y_train)}')

Training score with OHE and Random Forest is  0.9081313131313131


## Using Random Forest and RandomizedSearchCV using OrdinalEncoding

In [ ]:
#using Ordinal Encoding

#making the pipeline
rsearchcv_pipeline_OE = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_jobs=-1)
)
#defining the param_distribution for RandomizedSearchCV
param_distribution = {
    'simpleimputer__strategy':['mean', 'median'],
    'randomforestclassifier__n_estimators':range(100,500,100),
    'randomforestclassifier__max_depth':range(5,31,5),
    'randomforestclassifier__max_samples':np.arange(0.2,0.7,0.2),
}

search_model_OE = RandomizedSearchCV(
    rsearchcv_pipeline_OE,
    param_distributions=param_distribution,
    cv=5,
    n_jobs=-1,
    verbose=1,
    n_iter=10
)

In [ ]:
# fit the model
search_model_OE.fit(X_train_OE, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  3.3min finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('ordinalencoder',
                                              OrdinalEncoder()),
                                             ('simpleimputer', SimpleImputer()),
                                             ('randomforestclassifier',
                                              RandomForestClassifier(n_jobs=-1,
                                                                     random_state=42))]),
                   n_jobs=-1,
                   param_distributions={'randomforestclassifier__max_depth': range(5, 31, 5),
                                        'randomforestclassifier__max_samples': array([0.2, 0.4, 0.6]),
                                        'randomforestclassifier__n_estimators': range(100, 500, 100),
                                        'simpleimputer__strategy': ['mean',
                                                                    'median']},
                   verbose=1)

In [ ]:
#print the scores
#scores
print('Best Hyperparameters', search_model_OE.best_params_)
print(f'Best score with OE and Random Forest is : {search_model_OE.best_score_}')

Best Hyperparameters {'simpleimputer__strategy': 'mean', 'randomforestclassifier__n_estimators': 400, 'randomforestclassifier__max_samples': 0.4, 'randomforestclassifier__max_depth': 30}
Best score with OE and Random Forest is : 0.8129461279461279


In [ ]:
# assigning the best estimator values to the search_model_OHE
pipeline_OE = search_model_OE.best_estimator_
print(f'Training score with OHE and Random Forest is{pipeline_OE.score(X_train_OE,y_train)}')

Training score with OHE and Random Forest is0.9422558922558922


In [ ]:
#predict
y_pred_OE = pipeline_OE.predict(X_test_OE)
y_pred_OE

array(['non functional', 'functional', 'functional', ..., 'functional',
       'functional', 'non functional'], dtype=object)

## Submission

In [ ]:
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv',  index_col='id')
# YOUR CODE HER

submission = pd.DataFrame(pipeline_OHE.predict(X_test), index=test.index, columns=['status_group'])
submission.head()

,status_group
id,
50785,functional
51630,functional
17168,functional
45559,non functional
49871,functional


In [ ]:
# best case using OHE. Ran Ordinal Encoding but give a better prediction.
submission.to_csv('PS_DS_223_v2.csv')

In [ ]:
from sklearn.model_selection import cross_val_score
scores_rf = cross_val_score(pipeline_OHE, X_train, y_train, cv=5)
print(scores_rf)

/Users/rob/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/Users/rob/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/Users/rob/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/Users/rob/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/Users/rob/opt/anaco

[0.81447811 0.80909091 0.81439394 0.80765993 0.81043771]
0.8112121212121212


In [ ]:
print(scores_rf.mean())

0.8112121212121212
